In [1]:
from model import layers, metrics, loss
from model.slomo import SloMo
from dataset2 import *

In [8]:
import os
import pathlib
import tensorflow as tf
import config
import datetime

In [41]:
data_dir = ".\Adobe"
data_dir = Path(data_dir)
train_ds = load_dataset(os.path.join(data_dir, "train", "frames"), 32)

Adobe\train\frames


In [42]:
train_ds

<PrefetchDataset shapes: (((32, 352, 352, 3), (32, 352, 352, 3), (32,)), (32, 352, 352, 3)), types: ((tf.float32, tf.float32, tf.int32), tf.float32)>

In [43]:
for x in enumerate(train_ds):
    print(x)

UnknownError: FindFirstFile failed for: Adobe/train/frames/268/frame3220.jpg : The directory name is invalid.
; Unknown error
	 [[{{node MatchingFiles}}]]

In [28]:
def train(data_dir, model_dir, log_dir, epochs: int, batch_size: int):
    tf.keras.backend.clear_session()
    os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

    
    print("Eager execution: {}".format(tf.executing_eagerly()))
    
    #Loading dataset
    data_dir = pathlib.Path(data_dir)
    train_ds = load_dataset(os.path.join(data_dir, "train", "frames"), batch_size)
    len_train = tf.data.experimental.cardinality(train_ds).numpy()
    progbar = tf.keras.utils.Progbar(len_train)
    valid_ds = load_dataset(os.path.join(data_dir, "test", "frames"), batch_size, train = False)
    len_valid = tf.data.experimental.cardinality(valid_ds).numpy()
    val_progbar = tf.keras.utils.Progbar(len_valid)
    
    #Model checkpoint
    model_dir = Path(model_dir)
    model_dir.mkdir(parents = True, exist_ok = True)
    checkpoint = model_dir / "checkpoint"
    checkpoint.mkdir(parents = True, exist_ok = True)
    
    #Tensorboard
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = log_dir / "gradient_tape" / current_time / "train"
    test_log_dir = log_dir / "gradient_tape" / current_time / "test"
    train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
    test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))
    
    #Model
    model = SloMo()
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4)
    ckpt = tf.train.Checkpoint(step = tf.Variable(1), optimizer = optimizer, net = model)
    manager = tf.train.CheckpointManager(ckpt, str(checkpoint), max_to_keep = 3)
    num_epochs = 1
    
    if manager.latest_checkpoint:
        status = ckpt.restore(manager.latest_checkpoint).assert_nontrivial_match()
        print("Restored from {}.".format(manager.latest_checkpoint))
        num_epochs = int(manager.latest_checkpoint.split("-")[-1]) + 1
    else:
        print("No checkpoint provided, starting new train.")
        
    loss_obj = loss.Loss()
    
    for epoch in range(num_epochs, epochs):
        print("Epoch: " + str(epoch))
        
        avg_losses, avg_metrics = [0] * 5, [0] * 2
        avg_val_losses, avg_val_metrics = [0] * 5, [0] * 2
        
        print(train_ds)
        for step, frames in enumerate(train_ds):
            print(step, frames)
#             inputs, targets = frames
#             loss_values, metric_values = train_step(model, inputs, targets, optimizer, loss_obj)
            
#             avg_losses = [sum(x) for x in zip(avg_losses, loss_values)]
#             avg_metrics = [sum(x) for x in zip(avg_metrics, metric_values)]
#             progbar.update(step + 1,
#                           [
#                               ("Total_loss", loss_values[0]),
#                               ("Reconstruction_loss", loss_values[1]),
#                               ("Perceptual_loss", loss_values[2]),
#                               ("Smooth_loss", loss_values[3]),
#                               ("Warping_loss", loss_values[4]),
#                               ("PSNR", metric_values[0]),
#                               ("SSIM", metric_values[1])
#                           ])
            
#         avg_losses = [x / len(avg_losses) for x in avg_losses]
#         avg_metrics = [tf.reduce_mean(x) / len(avg_metrics) for x in avg_metrics]
#         with train_summary_writer.as_default():
#             tf.summary.scalar("Total_loss", avg_losses[0], step = epoch)
#             tf.summary.scalar("Reconstruction_loss", avg_losses[1], step = epoch)
#             tf.summary.scalar("Perceptual_loss", avg_losses[2], step = epoch)
#             tf.summary.scalar("Smooth_loss", avg_losses[3], step = epoch)
#             tf.summary.scalar("Warping_loss", avg_losses[4], step = epoch)
#             tf.summary.scalar("PSNR", metric_values[0], step = epoch)
#             tf.summary.scalar("SSIM", metric_values[1], step = epoch)
            
#         for step, frames in enumerate(valid_ds):
#             inputs, targets = frames
#             val_loss_values, val_metric_values = valid_step(model, inputs, targets, loss_obj)
            
#             avg_val_losses = [sum(x) for x in zip(avg_val_losses, val_loss_values)]
#             avg_val_metrics = [sum(x) for x in zip(avg_val_metrics, val_metric_values)]
#             val_progbar.update(step + 1,
#                               [
#                                   ("Total_loss", avg_val_losses[0]),
#                                   ("Reconstruction_loss", avg_val_losses[1]),
#                                   ("Perceptual_loss", avg_val_losses[2]),
#                                   ("Smooth_loss", avg_val_losses[3]),
#                                   ("Warping_loss", avg_val_losses[4]),
#                                   ("PSNR", avg_val_metrics[0]),
#                                   ("SSIM", avg_val_metrics[1])   
#                               ])
            
#         avg_val_losses = [x / len(avg_val_losses) for x in avg_val_losses]
#         avg_val_metrics = [tf.reduce_mean(x) / len(avg_val_metrics) for x in avg_val_metrics]
#         with test_summary_writer.as_default():
#             tf.summary.scalar("Val_total_loss", avg_val_losses[0], step = epoch)
#             tf.summary.scalar("Val_reconstruction_loss", avg_val_losses[1], step = epoch)
#             tf.summary.scalar("Val_perceptual_loss", avg_val_losses[2], step = epoch)
#             tf.summary.scalar("Val_smooth_loss", avg_val_losses[3], step = epoch)
#             tf.summary.scalar("Val_warping_loss", avg_val_losses[4], step = epoch)
#             tf.summary.scalar("Val_PSNR", avg_val_metrics[0], step = epoch)
#             tf.summary.scalar("Val_SSIM", avg_val_metrics[1], step = epoch)
        
#         ckpt.step.assign_add(1)
#         save_path = manager.save()
#         print("Save checkpoint for step {}: {}".format(int(ckpt.step), save_path))
    
#     final_file = model_dir / "weights_final_{}.tf".format(epochs)
#     model.save_weights(str(final_file), save_format = "tf")

In [29]:
@tf.function
def train_step(model, inputs, targets, optimizer, loss_obj):
    with tf.GradientTape() as tape:
        predictions, loss_outputs = model(inputs, training = True)
        loss_values = loss_obj.total_loss(predictions, loss_outputs, inputs, targets)
        metric_values = metrics.metrics(targets, predictions)
        
    gradient = tape.gradient(loss_values, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    return loss_values, metric_values

@tf.function
def valid_step(model, inputs, targets, loss_obj):
    predictions, loss_outputs = model(inputs, training = False)
    loss_values = loss_obj.total_loss(predictions, loss_outputs, inputs, targets)
    metric_values = metrics.metrics(targets, predictions)
    return loss_values, metric_values

In [30]:
def main():
    log_dir = Path("./")
    log_dir.mkdir(parents = True, exist_ok = True)
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = log_dir / current_time / "train"
    train_log_dir.mkdir(parents=True, exist_ok=True)
    
    data_dir = ".\Adobe"
    model_dir = "./"
    
    train(data_dir, model_dir, train_log_dir, epochs = 10, batch_size = 32)

In [31]:
if __name__ == "__main__":
    main()

Eager execution: True
Adobe\train\frames
Adobe\test\frames
No checkpoint provided, starting new train.
Epoch: 1
<PrefetchDataset shapes: (((32, 352, 352, 3), (32, 352, 352, 3), (32,)), (32, 352, 352, 3)), types: ((tf.float32, tf.float32, tf.int32), tf.float32)>


UnknownError: FindFirstFile failed for: Adobe/train/frames/176/frame2120.jpg : The directory name is invalid.
; Unknown error
	 [[{{node MatchingFiles}}]]